In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)

LR_tune,N_value = np.meshgrid(lr_tune,n_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)

lrn_tune = np.hstack((LR_tune,N_value))

In [6]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y
    

In [7]:
loss_thresh = 0.005

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [8]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(n_val)
        
              
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
       
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
      
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0

  x_coll = torch.from_numpy(colloc_pts(N_f,123)).float().to(device)
  f_hat = torch.zeros(x_coll.shape[0],1).to(device)

  for i in range(max_iter):
    
    train_step(i)

    loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
for tune_reps in range(25):

  label = "1D_FODE_atanh_tune"+str(tune_reps)  
  max_reps = 10
  max_iter = 100

  train_loss_full = []
  test_mse_full = []
  test_re_full = []
  alpha_full = []
  elapsed_time= np.zeros((max_reps,1))
  time_threshold = np.empty((max_reps,1))
  time_threshold[:] = np.nan
  epoch_threshold = max_iter*np.ones((max_reps,1))
  n_val = lrn_tune[tune_reps,1]

  for reps in range(max_reps):  
      print(label) 
      train_loss = []
      test_mse_loss = []
      test_re_loss = []   
      alpha_val = []
      
      torch.manual_seed(reps*36)
      N_f = 10000 #Total number of collocation points
    
      layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
      PINN = Sequentialmodel(layers,n_val)
      
    
      PINN.to(device)

      'Neural Network Summary'
      print(PINN)

      params = list(PINN.parameters())
      
      optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrn_tune[tune_reps,0], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-6, 
                                tolerance_change = 1e-6, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')
    

      
      train_model(max_iter,reps)

      
      torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
      train_loss_full.append(train_loss)
      test_mse_full.append(test_mse_loss)
      test_re_full.append(test_re_loss)
      alpha_full.append(alpha_val)
      
      
      print('Training time: %.2f' % (elapsed_time[reps]))

  mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full, "label": label}
  savemat(label+'.mat', mdic)

1D_FODE_atanh_tune0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.76838 Test MSE 5220.123714911537 Test RE 0.9997549487361271
1 Train Loss 47.768135 Test MSE 5219.870557190222 Test RE 0.9997307061350746
2 Train Loss 47.76793 Test MSE 5219.331939928068 Test RE 0.9996791257269652
3 Train Loss 47.755405 Test MSE 5175.943526429483 T

96 Train Loss 4.0366406 Test MSE 1081.2552472985233 Test RE 0.4550061625461254
97 Train Loss 3.997869 Test MSE 1071.6839189630766 Test RE 0.45298781674694344
98 Train Loss 3.9691389 Test MSE 1047.8180186134293 Test RE 0.4479155046898982
99 Train Loss 3.9648273 Test MSE 1041.9552228023167 Test RE 0.4466606490092494
Training time: 89.49
Training time: 89.49
1D_FODE_atanh_tune0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, 

92 Train Loss 22.13474 Test MSE 6266.917861254402 Test RE 1.0954187388921448
93 Train Loss 22.13474 Test MSE 6266.917861254402 Test RE 1.0954187388921448
94 Train Loss 22.13474 Test MSE 6266.917861254402 Test RE 1.0954187388921448
95 Train Loss 22.13474 Test MSE 6266.917861254402 Test RE 1.0954187388921448
96 Train Loss 22.134737 Test MSE 6266.917832193482 Test RE 1.0954187363523098
97 Train Loss 22.13467 Test MSE 6266.9239363328625 Test RE 1.0954192698352108
98 Train Loss 22.13467 Test MSE 6266.9239363328625 Test RE 1.0954192698352108
99 Train Loss 22.13467 Test MSE 6266.9239363328625 Test RE 1.0954192698352108
Training time: 61.59
Training time: 61.59
1D_FODE_atanh_tune0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=

89 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
90 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
91 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
92 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
93 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
94 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
95 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
96 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
97 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
98 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
99 Train Loss 37.51298 Test MSE 5143.580263153685 Test RE 0.9923981026292278
Training time: 31.59
Training time: 31.59
1D_FODE_atanh_tune0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleLis

85 Train Loss 23.333643 Test MSE 6752.94087794516 Test RE 1.1371025629975404
86 Train Loss 23.211132 Test MSE 6763.324070298201 Test RE 1.1379764207638063
87 Train Loss 23.181324 Test MSE 6794.769404384058 Test RE 1.1406188012907572
88 Train Loss 23.075 Test MSE 6810.367037070665 Test RE 1.1419272161759155
89 Train Loss 22.889984 Test MSE 6856.408146749993 Test RE 1.1457806822531618
90 Train Loss 22.649618 Test MSE 7032.403341656854 Test RE 1.1603928669620696
91 Train Loss 22.48621 Test MSE 7151.261809409536 Test RE 1.1701579934153352
92 Train Loss 22.464254 Test MSE 7149.216972495779 Test RE 1.169990683547751
93 Train Loss 22.421951 Test MSE 7177.583232396725 Test RE 1.172309497234702
94 Train Loss 22.382227 Test MSE 7263.5943078861355 Test RE 1.1793126430190475
95 Train Loss 22.371475 Test MSE 7306.906433440026 Test RE 1.1828234819478396
96 Train Loss 22.35728 Test MSE 7335.483982269341 Test RE 1.1851342553387736
97 Train Loss 22.347137 Test MSE 7355.845994943095 Test RE 1.1867779775

...

40 Train Loss 111.43284 Test MSE 5203.515711974701 Test RE 0.9981633044017697
41 Train Loss 111.28253 Test MSE 5203.517182719464 Test RE 0.9981634454644114
42 Train Loss 111.150276 Test MSE 5203.518515911062 Test RE 0.998163573333984
43 Train Loss 111.024124 Test MSE 5203.51823110376 Test RE 0.9981635460174406
44 Train Loss 110.953735 Test MSE 5203.517398993237 Test RE 0.9981634662077442
45 Train Loss 110.69897 Test MSE 5203.5138409847705 Test RE 0.998163124950629
46 Train Loss 110.58346 Test MSE 5203.506919381425 Test RE 0.9981624610827375
47 Train Loss 110.21718 Test MSE 5203.502705159497 Test RE 0.9981620568861979
48 Train Loss 109.82878 Test MSE 5203.502093146166 Test RE 0.9981619981864586
49 Train Loss 109.55404 Test MSE 5203.4994416848685 Test RE 0.9981617438780912
50 Train Loss 109.41182 Test MSE 5203.497141512048 Test RE 0.998161523262639
51 Train Loss 109.33736 Test MSE 5203.49600626697 Test RE 0.9981614143783647
52 Train Loss 109.169365 Test MSE 5203.496191463587 Test RE 0.99

36 Train Loss 133.04443 Test MSE 5294.070019612437 Test RE 1.006811123693314
37 Train Loss 132.95538 Test MSE 5294.068963292393 Test RE 1.0068110232493435
38 Train Loss 132.88832 Test MSE 5294.063581558857 Test RE 1.0068105115078019
39 Train Loss 132.83139 Test MSE 5294.058712645717 Test RE 1.006810048529413
40 Train Loss 132.72914 Test MSE 5294.048155274504 Test RE 1.0068090446424807
41 Train Loss 132.62852 Test MSE 5294.044883037841 Test RE 1.0068087334894762
42 Train Loss 132.50136 Test MSE 5294.042479782354 Test RE 1.0068085049667739
43 Train Loss 132.41664 Test MSE 5294.042343112112 Test RE 1.0068084919709614
44 Train Loss 132.35861 Test MSE 5294.043455954328 Test RE 1.0068085977898107
45 Train Loss 132.3391 Test MSE 5294.0375865661035 Test RE 1.0068080396765071
46 Train Loss 132.29523 Test MSE 5294.034535111066 Test RE 1.006807749517072
47 Train Loss 132.25261 Test MSE 5294.030997765026 Test RE 1.0068074131547027
48 Train Loss 132.22467 Test MSE 5294.029812231282 Test RE 1.006807

32 Train Loss 90.90449 Test MSE 5194.095473488644 Test RE 0.9972593774773112
33 Train Loss 90.707054 Test MSE 5194.10221962176 Test RE 0.9972600251013645
34 Train Loss 90.52235 Test MSE 5194.105785784022 Test RE 0.9972603674502736
35 Train Loss 90.38134 Test MSE 5194.1148903531175 Test RE 0.9972612414815621
36 Train Loss 90.31035 Test MSE 5194.11777817653 Test RE 0.9972615187101196
37 Train Loss 90.13437 Test MSE 5194.130566439781 Test RE 0.9972627463714403
38 Train Loss 90.0605 Test MSE 5194.134985847966 Test RE 0.9972631706301445
39 Train Loss 89.912476 Test MSE 5194.129703807078 Test RE 0.9972626635595545
40 Train Loss 89.81681 Test MSE 5194.125490653405 Test RE 0.9972622591008725
41 Train Loss 89.7606 Test MSE 5194.126288434425 Test RE 0.9972623356870925
42 Train Loss 89.73122 Test MSE 5194.127717826763 Test RE 0.9972624729073837
43 Train Loss 89.72123 Test MSE 5194.128344106035 Test RE 0.9972625330295861
44 Train Loss 89.66876 Test MSE 5194.130719050958 Test RE 0.9972627610219614


30 Train Loss 136.27385 Test MSE 5295.182718783649 Test RE 1.0069169231104336
31 Train Loss 136.048 Test MSE 5295.137044214898 Test RE 1.0069125804278625
32 Train Loss 135.88168 Test MSE 5295.119339367292 Test RE 1.0069108970673997
33 Train Loss 135.35506 Test MSE 5295.137849889561 Test RE 1.0069126570305942
34 Train Loss 134.84186 Test MSE 5295.043957614025 Test RE 1.006903729808828
35 Train Loss 134.64636 Test MSE 5294.999052372728 Test RE 1.0068994602171897
36 Train Loss 134.38103 Test MSE 5294.948777651911 Test RE 1.0068946800738021
37 Train Loss 133.70424 Test MSE 5294.749214255187 Test RE 1.0068757052711843
38 Train Loss 133.51543 Test MSE 5294.709759359139 Test RE 1.0068719537950819
39 Train Loss 133.39407 Test MSE 5294.671968656455 Test RE 1.0068683605417905
40 Train Loss 133.31636 Test MSE 5294.655853540858 Test RE 1.0068668282643929
41 Train Loss 133.28221 Test MSE 5294.646081424388 Test RE 1.0068658990987656
42 Train Loss 133.15506 Test MSE 5294.594612988998 Test RE 1.006861

27 Train Loss 70.164894 Test MSE 5218.925379898672 Test RE 0.9996401899498925
28 Train Loss 70.15775 Test MSE 5218.925466541483 Test RE 0.9996401982477343
29 Train Loss 69.88555 Test MSE 5218.928106418677 Test RE 0.999640451070566
30 Train Loss 69.84743 Test MSE 5218.928604290202 Test RE 0.9996404987520527
31 Train Loss 69.83107 Test MSE 5218.929433525274 Test RE 0.9996405781684417
32 Train Loss 69.783424 Test MSE 5218.930310161213 Test RE 0.9996406621244359
33 Train Loss 69.62264 Test MSE 5218.934227264379 Test RE 0.9996410372678692
34 Train Loss 69.52771 Test MSE 5218.938955293636 Test RE 0.9996414900740163
35 Train Loss 69.47966 Test MSE 5218.935754091413 Test RE 0.9996411834930162
36 Train Loss 69.47128 Test MSE 5218.936856497155 Test RE 0.9996412890710665
37 Train Loss 69.46594 Test MSE 5218.937240165918 Test RE 0.9996413258152481
38 Train Loss 69.459496 Test MSE 5218.937924609756 Test RE 0.9996413913648315
39 Train Loss 69.43239 Test MSE 5218.941872009709 Test RE 0.99964176940953

In [14]:
label

'1D_FODE_atanh_tune24'

In [15]:
#3,4,8,9,13,14,18,19,23,24

In [16]:
import scipy.io as sio

In [19]:
#s = set([3,4,8,9,13,14,18,19,23,24])
for tune_reps in range(25):
  #if tune_reps not in s:
    label = "1D_FODE_atanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.7098125977267593
1   0.38437629812690516
2   0.9823708885369958
3   0.997466702793887
4   nan
5   0.7885491632532698
6   0.3119089438304601
7   0.9683791637780045
8   0.9975815379364488
9   nan
10   0.7802964997745263
11   0.28846254655042347
12   0.9138193159014156
13   0.9986188978627473
14   nan
15   0.7540105507243202
16   0.22270447442310343
17   0.8700748059684509
18   0.9979353498430008
19   nan
20   0.5980961141199792
21   0.2539091229982887
22   0.8370243697174635
23   0.9969065990442036
24   nan


In [20]:
lrn_tune[16]

array([0.5, 3. ])